In [ ]:
# !pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"

In [21]:
import dlt
import requests
import json

1. dlt version

In [5]:
!dlt --version

dlt 1.12.3


In [8]:
@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc

In [9]:
from dlt.destinations import qdrant

qdrant_destination = qdrant(
  qd_path="db.qdrant", 
)

2. rows inserted into the zoomcamp_data collection

In [10]:
pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"

)
load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)

/home/l13/miniforge3/envs/llmzoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  2.93it/s]


Run started at 2025-07-08 13:59:20.859148+00:00 and COMPLETED in 11.41 seconds with 4 steps.
Step extract COMPLETED in 0.80 seconds.

Load package 1751983164.7956796 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.10 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1751983164.7956796 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.58 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.56 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /home/l13/REPO/llmzoomcamp/workshops/db.qdrant location to store data
Load package 1751983164.7956796 is LOADED and contains no failed jobs

Step run COMPLETED in 11.41 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.56 seconds
1 load package(s) were loaded to dest

3. Embedding model used

In [ ]:
# Read the data and print model used from qdrant json file
with open('db.qdrant/meta.json', 'r') as f:
    meta = json.load(f)

print(meta['collections']['zoomcamp_tagged_data']['vectors'].keys())

dict_keys(['fast-bge-small-en'])
